In [ ]:
import torch 
from torch import nn,optim
from torch.utils.data import(Dataset, DataLoader, TensorDataset)
import torch.nn.functional as F
import tqdm
import re 
import collections
import itertools
import MeCab
import neologdn
import math
import emoji
import matplotlib.pyplot as plt

import math
import torchtext
from torchtext.vocab import Vectors
import glob
import os
import io
import string
import re
import random
from time import sleep
import csv
import pandas as pd
import torchnlp.metrics.bleu as bleu

mecab = MeCab.Tagger('-Owakati -d /usr/lib/x86_64-linux-gnu/mecab/dic/mecab-ipadic-neologd')
mecab.parse('')  # バグ対処

In [2]:
remove_marks_regex = re.compile("[\,\(\)\[\]\*:;]|<.*?>")
shift_marks_regex = re.compile("([?!\.])")

unk = 0
sos = 1
eos = 2

def normalize(text):
    text = text.lower()
#     #不要な文字を削除
#     text = remove_marks_regex.sub("", text)
    #?!.と単語の間に空白を挿入
    text = shift_marks_regex.sub(r"\1", text)
    #重ね表現の削除
    text = neologdn.normalize(text)
    #url削除
    text = re.sub(r'https?://[\w/:%#\$&\?\(\)~\.=\+\-]+', '', text)
    #絵文字削除
    text = ''.join(['' if c in emoji.UNICODE_EMOJI else c for c in text])
    #桁区切りの削除
    text = re.sub(r'(\d)([,.])(\d+)', r'\1\3', text)
    text = re.sub(r'\d+', '0', text)
    # 半角記号の置換
    text = re.sub(r'[!-/:-@[-`{-~]', r' ', text)
    # 全角記号の置換 (ここでは0x25A0 - 0x266Fのブロックのみを除去)
    text = re.sub(u'[■-♯]', ' ', text)
    text = text.replace("【", " ").replace("】", " ").replace("『"," ").replace("』", " ").replace("、", " ").replace("。", " ").replace("”", " ").replace('"', " ")
    return text

def make_wakati(sentence):
    sentence = sentence.lower()
    # MeCabで分かち書き
    sentence = mecab.parse(sentence)
    # 半角全角英数字除去
    sentence = re.sub(r'[0-9０-９a-zA-Zａ-ｚＡ-Ｚ]+', " ", sentence)
    # 記号もろもろ除去
    sentence = re.sub(r'[\．_－―─！＠＃＄％＾＆\-‐|\\＊\“（）＿■×+α※÷⇒—●★☆〇◎◆▼◇△□(：〜～＋=)／*&^%$#@!~`){}［］…\[\]\"\'\”\’:;<>?＜＞〔〕〈〉？、。・,\./『』【】「」→←○《》≪≫\n\u3000]+', "", sentence)
    # スペースで区切って形態素の配列へ
    wakati = sentence.split(" ")
    # 空の要素は削除
    wakati = list(filter(("").__ne__, wakati))
    return wakati

def parse_line(line):
    src,trg = line.split("\t")#[:2]
    #翻訳元と翻訳先それぞれのトークンリストを作成する
#     src = mecab.parse(str(src))
#     trg = mecab.parse(str(trg))
#     src = make_wakati(str(src))
#     trg = make_wakati(str(trg))
    src_tokens = src.strip().split()
    trg_tokens = trg.strip().split()
    return src_tokens, trg_tokens
#     return src_tokens, trg_tokens

def build_vocab(tokens):
    #ファイル中のすべての文章でのトークン数を数える
    counts = collections.Counter(tokens)
    #トークンの出現数の多い順に並べる
    sorted_counts = sorted(counts.items(), key=lambda c: c[1], reverse=True)
    #3つのタグを追加して正引きリストと逆引き用辞書を作る
    word_list = ["<UNK>", "<SOS>", "<EOS>"] + [x[0] for x in sorted_counts]
    word_dict = dict((w, i) for i, w in enumerate(word_list))
    return word_list, word_dict

def words2tensor(words, word_dict, max_len, padding=0):
    #末尾に終了タグをつける
    words = words + ["<EOS>"]
    #辞書を利用して数値のリストに変換する
    words = [word_dict.get(w,0) for w in words]
    seq_len = len(words)
    #長さがmax_len以下の場合はパディングする
    if seq_len < max_len + 1:
        words = words + [padding] * (max_len + 1 - seq_len)
    #Tensorに変換して返す
    return torch.tensor(words, dtype=torch.int64), seq_len

In [3]:
class TransformerModel(nn.Module):

    def __init__(self, ntoken, ninp, nhead, nhid, nlayers, dropout=0.5):
        super(TransformerModel, self).__init__()
        from torch.nn import TransformerEncoder, TransformerEncoderLayer
        self.model_type = 'Transformer'
        self.src_mask = None
        self.pos_encoder = PositionalEncoding(ninp, dropout)
        encoder_layers = TransformerEncoderLayer(ninp, nhead, nhid, dropout)
        self.transformer_encoder = TransformerEncoder(encoder_layers, nlayers)
        self.encoder = nn.Embedding(ntoken, ninp)
        self.ninp = ninp
        self.decoder = nn.Linear(ninp, ntoken)

        self.init_weights()

    def _generate_square_subsequent_mask(self, sz):
        mask = (torch.triu(torch.ones(sz, sz)) == 1).transpose(0, 1)
        mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
        return mask

    def init_weights(self):
        initrange = 0.1
        self.encoder.weight.data.uniform_(-initrange, initrange)
        self.decoder.bias.data.zero_()
        self.decoder.weight.data.uniform_(-initrange, initrange)

    def forward(self, src):
        if self.src_mask is None or self.src_mask.size(0) != src.size(0):
            device = src.device
            mask = self._generate_square_subsequent_mask(src.size(0)).to(device)
            self.src_mask = mask

        src = self.encoder(src) * math.sqrt(self.ninp)
        src = self.pos_encoder(src)
        output = self.transformer_encoder(src, self.src_mask)
        #output = self.decoder(output)
        #######
        #outputを線形層に置き換え
        #######
        
        return output

class PositionalEncoding(nn.Module):

    def __init__(self, d_model, dropout=0.1, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:x.size(0), :]
        return self.dropout(x)

In [4]:
remove_marks_regex = re.compile("[\,\(\)\[\]\*:;]|<.*?>")
shift_marks_regex = re.compile("([?!\.])")

def get_DataLoaders_and_TEXT(max_length=256, batch_size=24):
    """IMDbのDataLoaderとTEXTオブジェクトを取得する。 """
    def normalize(text):
        text = text.lower()
        #不要な文字を削除
        text = remove_marks_regex.sub("", text)
        #?!.と単語の間に空白を挿入
        text = shift_marks_regex.sub(r"\1", text)
        #重ね表現の削除
        text = neologdn.normalize(text)
        #url削除
        text = re.sub(r'https?://[\w/:%#\$&\?\(\)~\.=\+\-]+', '', text)
        #絵文字削除
        text = ''.join(['' if c in emoji.UNICODE_EMOJI else c for c in text])
        #桁区切りの削除
        text = re.sub(r'(\d)([,.])(\d+)', r'\1\3', text)
        text = re.sub(r'\d+', '0', text)
        # 半角記号の置換
        text = re.sub(r'[!-/:-@[-`{-~]', r' ', text)
        # 全角記号の置換 (ここでは0x25A0 - 0x266Fのブロックのみを除去)
        text = re.sub(u'[■-♯]', ' ', text)
        text = text.replace("【", "「").replace("】", "」").replace("『","「").replace("』","」")
        return text

    def preprocessing_text(text):
        # 改行コードを消去
        text = re.sub('<br />', '', text)

        # カンマ、ピリオド以外の記号をスペースに置換
        for p in string.punctuation:
            if (p == ".") or (p == ","):
                continue
            else:
                text = text.replace(p, " ")

        # ピリオドなどの前後にはスペースを入れておく
        text = text.replace(".", " . ")
        text = text.replace(",", " , ")
        text = text.replace("・", " ・ ")
        text = text.replace("、", "　、　")
        text = text.replace("。", "　。　")
        text = text.replace("「", " 　「　")
        text = text.replace("」", " 　」　")
        text = text.replace("【", "　【　")
        text = text.replace("】", "　】　")
        text = text.replace('"', '　"　')
        text = text.replace("'", "　'　")
        return text

    # 分かち書き
    def tokenizer_punctuation(text):
        text = normalize(text)
        text = preprocessing_text(text)
        return text


    # 前処理と分かち書きをまとめた関数を定義
    def tokenizer_with_preprocessing(text):
        text = preprocessing_text(text)
        ret = tokenizer_punctuation(text)
        #mecabで分かち書き
        ret = mecab.parse(str(ret))
        ret = ret.strip().split()
        return ret    
    
    
    
    # データを読み込んだときに、読み込んだ内容に対して行う処理を定義します
    # max_length
    TEXT = torchtext.data.Field(sequential=True, tokenize=tokenizer_with_preprocessing, use_vocab=True,
                                lower=True, include_lengths=True, batch_first=True, fix_length=100, init_token="<cls>", eos_token="<eos>")
    LABEL = torchtext.data.Field(sequential=True, tokenize=tokenizer_with_preprocessing, use_vocab=True,
                                lower=True, include_lengths=True, batch_first=True, fix_length=100, unk_token="<unk>")

    # フォルダ「data」から各tsvファイルを読み込みます
    train_val_ds, test_ds = torchtext.data.TabularDataset.splits(
        path='./train_data', train='Text.tsv',
        test='Text.tsv', format='tsv',
        fields=[('Text', TEXT), ('Label', LABEL)])

    # torchtext.data.Datasetのsplit関数で訓練データとvalidationデータを分ける
    train_ds, val_ds = train_val_ds.split(
        split_ratio=0.8, random_state=random.seed(1234))

    # torchtextで単語ベクトルとして読み込む(日本語)
#     japanese_fasttext_vectors = Vectors(name = ".vec")

    # ベクトル化したバージョンのボキャブラリーを作成します
    TEXT.build_vocab(train_ds, min_freq=1)
    LABEL.build_vocab(train_ds, min_freq=1)
    
    # DataLoaderを作成します（torchtextの文脈では単純にiteraterと呼ばれています）
    train_dl = torchtext.data.Iterator(
        train_ds, batch_size=batch_size, train=True)

    val_dl = torchtext.data.Iterator(
        val_ds, batch_size=batch_size, train=False, sort=False)

    test_dl = torchtext.data.Iterator(
        test_ds, batch_size=batch_size, train=False, sort=False)

    return train_dl, val_dl, test_dl, TEXT, LABEL

In [7]:
glob.glob("./*")

['./catboost_info',
 './test.csv',
 './Attention_LSTM_LCM.ipynb',
 './train.dat',
 './Untitled2.ipynb',
 './README.md',
 './Untitled1.ipynb',
 './train.csv',
 './data.csv',
 './voc.txt']

In [ ]:
train_dl, val_dl, test_dl, TEXT, LABEL = get_DataLoaders_and_TEXT()

In [ ]:
 dataloders_dict = {"train": train_dl, "val": val_dl}